In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# класс источника событий
class EventSource:
    def __init__(self, name):
        self.name = name
        self.subscribers = []

    def add_subscriber(self, subscriber):
        if subscriber not in self.subscribers:
            self.subscribers.append(subscriber)
            self.notify_subscribers(f"Subscriber {subscriber.name} added.")

    def remove_subscriber(self, subscriber):
        if subscriber in self.subscribers:
            self.subscribers.remove(subscriber)
            self.notify_subscribers(f"Subscriber {subscriber.name} removed.")

    def notify_subscribers(self, message):
        for subscriber in self.subscribers:
            subscriber.notify(message, self.name)
    
    def list_of_subscribers(self):
        print(f"Subscribers of {self.name}: {[subscriber.name for subscriber in self.subscribers]}")

# класс подписчика
class Subscriber:
    def __init__(self, name):
        self.name = name

    def notify(self, message, event_source_name):
        print(f"{self.name} received message '{message}' from event source '{event_source_name}'")


def change_event_source(subscriber, new_event_source, event_sources):
    old_event_source = None

    add_to_new_source = list(event_source for event_source in event_sources if event_source == new_event_source)
    remove_from_old_source = list(event_source for event_source in event_sources if subscriber in event_source.subscribers)

    try:
        # у нас транзакционная смена источника, поэтому нужно чтобы подписчик был в старом
        # и чтобы его не было в новом
        if remove_from_old_source and subscriber in remove_from_old_source[0].subscribers:
            for event_source in add_to_new_source:
                if subscriber not in event_source.subscribers:
                    remove_from_old_source[0].remove_subscriber(subscriber)
                    event_source.add_subscriber(subscriber)

                    # если успешно, обновляем переменные
                    old_event_source = remove_from_old_source[0]
        else:
            raise RuntimeError("Subscriber is not subscribed to the specified old event source.")

    except Exception as e:
        raise RuntimeError(f"Error during change_event_source: {str(e)}")

    return old_event_source, new_event_source


In [3]:
# создаем список источников событий
event_sources = [EventSource("EventSource1"), EventSource("EventSource2")]

# создаем подписчика
subscriber = Subscriber("Subscriber1")

# выводим лист подписчиков каждого из ресурсов (должны быть пустыми)
for event_source in event_sources:
    event_source.list_of_subscribers()
    
# подписываем подписчика на первый источник событий
event_sources[0].add_subscriber(subscriber)

# выводим лист подписчиков каждого из ресурсов после первого добавления
for event_source in event_sources:
    event_source.list_of_subscribers()

# меняем источник событий для подписчика с первого на второй
old_source, new_source = change_event_source(subscriber, event_sources[1], event_sources)

# выводим лист подписчиков каждого из ресурсов после смены
for event_source in event_sources:
    event_source.list_of_subscribers()

Subscribers of EventSource1: []
Subscribers of EventSource2: []
Subscriber1 received message 'Subscriber Subscriber1 added.' from event source 'EventSource1'
Subscribers of EventSource1: ['Subscriber1']
Subscribers of EventSource2: []
Subscriber1 received message 'Subscriber Subscriber1 added.' from event source 'EventSource2'
Subscribers of EventSource1: []
Subscribers of EventSource2: ['Subscriber1']


In [4]:
# теперь пытаемся сменить источник подписчика, когда он подписан на оба источника

# создаем подписчика
subscriber2 = Subscriber("SubscriberF1")

# подписываем подписчика на оба источника событий
event_sources[0].add_subscriber(subscriber2)
event_sources[1].add_subscriber(subscriber2)

# пытаемся изменить источник событий
try:
    old_source, new_source = change_event_source(subscriber2, event_sources[1], event_sources)
except RuntimeError as e:
    print(e)

# должно вывести, что он все еще подписан на оба
for event_source in event_sources:
    event_source.list_of_subscribers()

SubscriberF1 received message 'Subscriber SubscriberF1 added.' from event source 'EventSource1'
Subscriber1 received message 'Subscriber SubscriberF1 added.' from event source 'EventSource2'
SubscriberF1 received message 'Subscriber SubscriberF1 added.' from event source 'EventSource2'
Subscribers of EventSource1: ['SubscriberF1']
Subscribers of EventSource2: ['Subscriber1', 'SubscriberF1']


In [5]:
# теперь пытаемся сменить источник подписчика, когда он не подписан ни на один источник

# создаем подписчика
subscriber3 = Subscriber("SubscriberF2")

# пытаемся изменить источник событий
try:
    old_source, new_source = change_event_source(subscriber3, event_sources[1], event_sources)
except RuntimeError as e:
    print(e)

# SubscriberF2 не должно быть ни в одном из ресурсов
for event_source in event_sources:
    event_source.list_of_subscribers()

Error during change_event_source: Subscriber is not subscribed to the specified old event source.
Subscribers of EventSource1: ['SubscriberF1']
Subscribers of EventSource2: ['Subscriber1', 'SubscriberF1']
